# Week 1: This notebook will be used for my Capstone Project
##### Author: Amy TSE

In [3]:
import pandas as pd
import numpy as np

print("Hello Capstone Project Course!")

Hello Capstone Project Course!


# Week 3: Segmenting and Clustering Neighborhoods in Toronto

<a id='toc'></a>


## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#part1">Part 1: Scraping Data from Wiki</a>

2. <a href="#part2">Part 2: Getting Coordinates</a>

3. <a href="#part3">Part 3: Explore Toronto</a>

4. <a href="#part4">Part 4: Analyze Each Neighborhood in Toronto</a>

4. <a href="#part5">Part 5: Cluster Neighborhoods</a>

5. <a href="#part6">Part 6: Examine Clusters</a>  

</font>
</div>

<a id='part1'></a>



### Part 1: Scraping Data from Wiki 

Downloading all dependencies

In [113]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


#### Scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [84]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
df.head(15)

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


Three columns: PostalCode, Borough, and Neighborhood

In [85]:
#Check initial shape
df.shape

(180, 3)

Drop cells with a borough that is Not assigned. Sort and reset index

In [100]:
df.drop(df[df.Borough == 'Not assigned'].index, inplace=True)
df.sort_values(by=['Borough'], inplace = True)
df.reset_index(drop=True, inplace=True)
df.head(15)

,Postal Code,Borough,Neighborhood
0,M4T,Central Toronto,"Moore Park, Summerhill East"
1,M5N,Central Toronto,Roselawn
2,M4S,Central Toronto,Davisville
3,M5P,Central Toronto,"Forest Hill North & West, Forest Hill Road Park"
4,M5R,Central Toronto,"The Annex, North Midtown, Yorkville"
5,M4N,Central Toronto,Lawrence Park
6,M4P,Central Toronto,Davisville North
7,M4R,Central Toronto,"North Toronto West, Lawrence Park"
8,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest..."
9,M4W,Downtown Toronto,Rosedale


In [87]:
#shape after removing not assigned Borough
df.shape

(103, 3)

For the same postal code area, combine neighborhoods separated with a comma as one row

In [101]:
df.groupby(['Postal Code','Borough'])['Neighborhood'].apply(','.join).reset_index()
df.head(15)

,Postal Code,Borough,Neighborhood
0,M4T,Central Toronto,"Moore Park, Summerhill East"
1,M5N,Central Toronto,Roselawn
2,M4S,Central Toronto,Davisville
3,M5P,Central Toronto,"Forest Hill North & West, Forest Hill Road Park"
4,M5R,Central Toronto,"The Annex, North Midtown, Yorkville"
5,M4N,Central Toronto,Lawrence Park
6,M4P,Central Toronto,Davisville North
7,M4R,Central Toronto,"North Toronto West, Lawrence Park"
8,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest..."
9,M4W,Downtown Toronto,Rosedale


Wiki table already comes with that format, so the shape is the same

In [89]:
df.shape

(103, 3)

If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [102]:
#query rows with Not assigned neighborhood
df.loc[df['Neighborhood'] == 'Not assigned']

,Postal Code,Borough,Neighborhood


All Neighborhood with named Borough are named

Final shape

In [92]:
df.shape

(103, 3)

<a id='part2'></a>

<a href="#toc">Return to table of contents</a>

### Part 2: Getting Coordinates 

Getting coordinates from csv

In [76]:
#!wget -q -O 'toronto_data.json' http://cocl.us/Geospatial_data
import wget
filename = wget.download('http://cocl.us/Geospatial_data')
filename

100% [................................................................................] 2891 / 2891

'Geospatial_Coordinates.csv'

Load the data

In [78]:
df_cords = pd.read_csv(filename)
df_cords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merge the coordinates data with original dataframe

In [108]:
neighborhoods = pd.merge(df, df_cords, on=['Postal Code'])
neighborhoods.head(20)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
1,M5N,Central Toronto,Roselawn,43.711695,-79.416936
2,M4S,Central Toronto,Davisville,43.704324,-79.388790
3,M5P,Central Toronto,"Forest Hill North & West, Forest Hill Road Park",43.696948,-79.411307
4,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678
5,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
6,M4P,Central Toronto,Davisville North,43.712751,-79.390197
7,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678
8,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049
9,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529


Make sure no duplicates

In [109]:
neighborhoods.drop_duplicates(subset =None, 
                     keep = False, inplace = True)
neighborhoods.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
1,M5N,Central Toronto,Roselawn,43.711695,-79.416936
2,M4S,Central Toronto,Davisville,43.704324,-79.388790
3,M5P,Central Toronto,"Forest Hill North & West, Forest Hill Road Park",43.696948,-79.411307
4,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678


In [110]:
neighborhoods.shape

(103, 5)

<a id='part3'></a>

<a href="#toc">Return to table of contents</a>

### Part 3: Explore Toronto

Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

#### Use geopy library to get the latitude and longitude values of Toronto

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>to_explorer</em>, as shown below.

In [116]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### Create a map of Toronto with neighborhoods superimposed on top

In [123]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Simplifying the above map and segment and cluster only the neighborhoods in Old Toronto (with Keyword 'Toronto')

In [127]:
toronto_data = neighborhoods[neighborhoods['Borough'].str.contains("Toronto")].reset_index(drop=True)
toronto_data.head(5)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
1,M5N,Central Toronto,Roselawn,43.711695,-79.416936
2,M4S,Central Toronto,Davisville,43.704324,-79.388790
3,M5P,Central Toronto,"Forest Hill North & West, Forest Hill Road Park",43.696948,-79.411307
4,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678


In [128]:
toronto_data.shape

(39, 5)

Visualize Old Toronto only map

In [125]:
# create map of Old Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Define Foursquare Credentials and Version
Utilizing the Foursquare API to explore the neighborhoods and segment them.

In [129]:
CLIENT_ID = 'H0HHIXTTWHXYCPYGB1B1N5FKSQCCXCXPY5AIRTZEBKVDTC31' # your Foursquare ID
CLIENT_SECRET = 'XNAON0M3HTWMXBUYGQW0R2PRZWUDAYFIEJ5QPVCO5M3GS1FR' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

Explore <em><strong>one</strong></em> neighborhood in Old Toronto. Get neighborhood(s) name and coordinates

In [130]:
toronto_data.loc[0, 'Neighborhood']

'Moore Park, Summerhill East'

In [131]:
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Moore Park, Summerhill East are 43.6895743, -79.38315990000001.


Get the top 100 venues that are in Moore Park/ Summerhill East within a radius of 0.5km

In [143]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=H0HHIXTTWHXYCPYGB1B1N5FKSQCCXCXPY5AIRTZEBKVDTC31&client_secret=XNAON0M3HTWMXBUYGQW0R2PRZWUDAYFIEJ5QPVCO5M3GS1FR&v=20180605&ll=43.6895743,-79.38315990000001&radius=500&limit=100'

GET and find where all the informations are at

In [145]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ed9d3e01d67cb002825a98f'},
 'response': {'headerLocation': 'Moore Park',
  'headerFullLocation': 'Moore Park, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.6940743045, 'lng': -79.37694824615232},
   'sw': {'lat': 43.685074295499994, 'lng': -79.3893715538477}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '56952a8c498ecea4e5374f2d',
       'name': 'Camp Tournesol - Toronto Centre',
       'location': {'address': '1 Garfield Ave',
        'lat': 43.6868824390607,
        'lng': -79.38358068466187,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.6868824390607,
          'lng': -79.38358068466187}],
        'distance': 301,
        'postalCode': 

In [146]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Clean the json and structure into <em>pandas</em> dataframe

In [147]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(100)

,name,categories,lat,lng
0,Camp Tournesol - Toronto Centre,Summer Camp,43.686882,-79.383581
1,Ravine,Trail,43.690356,-79.386841
2,Totum Life Science St. Clair,Gym,43.686525,-79.383449
3,Reds,Restaurant,43.692816,-79.384504


In [148]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


Explore <em><strong>ALL</strong></em> neighborhood in Old Toronto. Get neighborhood(s) name and coordinates

In [149]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Create a new dataframe called toronto_venues

In [150]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Moore Park, Summerhill East
Roselawn
Davisville
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
Lawrence Park
Davisville North
North Toronto West, Lawrence Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
St. James Town
St. James Town, Cabbagetown
Kensington Market, Chinatown, Grange Park
University of Toronto, Harbord
Queen's Park, Ontario Provincial Government
Christie
Harbourfront East, Union Station, Toronto Islands
Stn A PO Boxes
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
First Canadian Place, Underground city
Business reply mail Processing Centre, South Central Letter Processing Plant Toronto
Studio District
India Bazaar, The Beaches West
The Danforth Wes

Check the size and partial content of <em>dataframe</em>

In [151]:
toronto_venues.shape

(1614, 7)

In [152]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Moore Park, Summerhill East",43.689574,-79.383160,Camp Tournesol - Toronto Centre,43.686882,-79.383581,Summer Camp
1,"Moore Park, Summerhill East",43.689574,-79.383160,Ravine,43.690356,-79.386841,Trail
2,"Moore Park, Summerhill East",43.689574,-79.383160,Totum Life Science St. Clair,43.686525,-79.383449,Gym
3,"Moore Park, Summerhill East",43.689574,-79.383160,Reds,43.692816,-79.384504,Restaurant
4,Roselawn,43.711695,-79.416936,Ceiling Champions,43.713891,-79.420702,Home Service


Check how many venues were returned for each neighborhood

In [153]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,54,54,54,54,54,54
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",19,19,19,19,19,19
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,63,63,63,63,63,63
Christie,17,17,17,17,17,17
Church and Wellesley,77,77,77,77,77,77
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,34,34,34,34,34,34


In [155]:
print('There are {} uniques Toronto Venue categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 236 uniques Toronto Venue categories.


<a id='part4'></a>

<a href="#toc">Return to table of contents</a>

### Part 4: Analyze Each Neighborhood in Toronto

Use onehot encoding

In [156]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Moore Park, Summerhill East",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [157]:
toronto_onehot.shape

(1614, 236)

Group neighborhood by mean of frequency of occurences in each category

In [158]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Berczy Park,0.000000,0.000000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.00,0.018519,0.000000,0.000000,0.000000,0.000000,0.000000,0.018519,0.000000,0.018519,0.037037,0.000000,0.000000,0.00,0.018519,0.018519,0.000000,0.037037,0.000000,0.000000,0.018519,0.0000,0.000000,0.000000,0.000000,0.018519,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.037037,0.00,0.000000,0.000000,0.000000,0.037037,0.000000,0.000000,0.000000,0.000000,0.018519,0.055556,0.074074,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.018519,0.000000,0.018519,0.000000,0.00,0.000000,0.000,0.018519,0.000000,0.000000,0.000000,0.0000

New size

In [159]:
toronto_grouped.shape

(39, 236)

Top 5 most common venues for each Neighborhood

In [160]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
          venue  freq
0   Coffee Shop  0.07
1  Cocktail Bar  0.06
2   Cheese Shop  0.04
3        Bakery  0.04
4    Restaurant  0.04


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0            Café  0.13
1       Nightclub  0.09
2  Breakfast Spot  0.09
3     Coffee Shop  0.09
4    Intersection  0.04


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                venue  freq
0  Light Rail Station  0.11
1         Yoga Studio  0.05
2             Brewery  0.05
3                 Spa  0.05
4          Skate Park  0.05


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                 venue  freq
0      Airport Service  0.19
1       Airport Lounge  0.12
2     Airport Terminal  0.12
3                Plane  0.06
4  Rental Car Location  0.06


----Central Bay Street----
                 venue  freq
0          Coffee Shop  0.17
1       

Put the above into <em>pandas</em> dataframe. Changing to top 10.

In [161]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [163]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Bakery,Beer Bar,Cheese Shop,Café,Restaurant,Japanese Restaurant,Beach
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Nightclub,Coffee Shop,Furniture / Home Store,Burrito Place,Restaurant,Italian Restaurant,Stadium,Intersection
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Gym / Fitness Center,Garden,Restaurant,Recording Studio,Pizza Place,Park,Garden Center,Fast Food Restaurant,Spa
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Sculpture Garden,Airport,Airport Food Court,Harbor / Marina,Boutique,Bar,Boat or Ferry
4,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Salad Place,Bubble Tea Shop,Burger Joint,Japanese Restaurant,Department Store,Modern European Restaurant


<a id='part5'></a>

<a href="#toc">Return to table of contents</a>

### Part 5: Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [165]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3])

Merge clusters and top 10 venues

In [166]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,4,Gym,Restaurant,Trail,Summer Camp,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run
1,M5N,Central Toronto,Roselawn,43.711695,-79.416936,3,Ice Cream Shop,Home Service,Garden,Music Venue,Dessert Shop,Diner,Discount Store,Distribution Center,Women's Store,Department Store
2,M4S,Central Toronto,Davisville,43.704324,-79.388790,3,Pizza Place,Sandwich Place,Dessert Shop,Sushi Restaurant,Coffee Shop,Gym,Italian Restaurant,Café,Thai Restaurant,Brewery
3,M5P,Central Toronto,"Forest Hill North & West, Forest Hill Road Park",43.696948,-79.411307,2,Park,Jewelry Store,Trail,Sushi Restaurant,Deli / Bodega,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant
4,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,3,Sandwich Place,Café,Coffee Shop,Pub,History Museum,Burger Joint,Indian Restaurant,Middle Eastern Restaurant,Cheese Shop,BBQ Joint


Visualize Clusters

In [167]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='part6'></a>

<a href="#toc">Return to table of contents</a>

### Part 6: Examine Clusters

There are 5 clusters. Confirming below:

In [168]:
print('There are {} clusters.'.format(len(toronto_merged['Cluster Labels'].unique())))

There are 5 clusters.


#### CLUSTER 1

In [169]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, 
                   toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Central Toronto,0,Pub,Coffee Shop,Sports Bar,Light Rail Station,Supermarket,Vietnamese Restaurant,Liquor Store,Sushi Restaurant,Bank,American Restaurant
32,East Toronto,0,Trail,Health Food Store,Pub,Doner Restaurant,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Women's Store


#### CLUSTER 2

In [170]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, 
                   toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Central Toronto,1,Park,Bus Line,Swim School,Women's Store,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant


#### CLUSTER 3

In [171]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, 
                   toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Central Toronto,2,Park,Jewelry Store,Trail,Sushi Restaurant,Deli / Bodega,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant
9,Downtown Toronto,2,Park,Playground,Trail,Dance Studio,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run


#### CLUSTER 4

In [172]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, 
                   toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Central Toronto,3,Ice Cream Shop,Home Service,Garden,Music Venue,Dessert Shop,Diner,Discount Store,Distribution Center,Women's Store,Department Store
2,Central Toronto,3,Pizza Place,Sandwich Place,Dessert Shop,Sushi Restaurant,Coffee Shop,Gym,Italian Restaurant,Café,Thai Restaurant,Brewery
4,Central Toronto,3,Sandwich Place,Café,Coffee Shop,Pub,History Museum,Burger Joint,Indian Restaurant,Middle Eastern Restaurant,Cheese Shop,BBQ Joint
6,Central Toronto,3,Gym / Fitness Center,Hotel,Pizza Place,Department Store,Sandwich Place,Breakfast Spot,Food & Drink Shop,Park,Gastropub,General Travel
7,Central Toronto,3,Clothing Store,Coffee Shop,Gym / Fitness Center,Italian Restaurant,Shoe Store,Salon / Barbershop,Restaurant,Rental Car Location,Park,Mexican Restaurant
10,Downtown Toronto,3,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Gay Bar,Restaurant,Yoga Studio,Mediterranean Restaurant,Café,Pub,Burger Joint
11,Downtown Toronto,3,Coffee Shop,Pub,Bakery,Park,Breakfast Spot,Café,Theater,Gym / Fitness Center,Event Space,Restaurant
12,Downtown Toronto,3,Clothing Store,Coffee Shop,Italian Restaurant,Café,Cosmetics Shop,Japanese Restaurant,Bubble Tea Shop,Middle Eastern Restaurant,Diner,Bakery
13,Downtown Toronto,3,Coffee Shop,Cocktail Bar,Seafood Restaurant,Bakery,Beer Bar,Cheese Shop,Café,Restaurant,Japanese Restaurant,Beach
14,Downtown Toronto,3,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Salad Place,Bubble Tea Shop,Burger Joint,Japanese Restaurant,Department Store,Modern European Restaurant


#### CLUSTER 5

In [174]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, 
                   toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,4,Gym,Restaurant,Trail,Summer Camp,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run


#### Final Summary
Basically cluster 1 is for Nightlife, cluster 2 and 3 are outdoor activities, cluster 4 the most popular ones - FOOD and coffee shops, cluster 5 being gym.
